In [ ]:
from os.path import isdir, join
from os import listdir 
from functools import partial
from warnings import warn
import numpy as np
from research_pyutils.path_related_functions import mkdir_p

# menpo packages imports
import menpo.io as mio
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector
from menpodetect.dlib.conversion import pointgraph_to_rect
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.landmark import LandmarkGroup

In [ ]:
path_base = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/300vw_testset/'
assert(isdir(path_base))

path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

In [ ]:
def return_detector(detection):
    if detection == 'dlib':
        return load_dlib_frontal_face_detector()
    elif detection == 'opencv':
        return load_opencv_frontal_face_detector()
    elif detection == 'ffld2':
        return load_ffld2_frontal_face_detector()
    else:
        raise RuntimeError('Not a valid choice of detection ({}).'.format(detection))
        
def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))

In [ ]:
method_detection = 'dlib'
method_landm_loc = 'dlibERT'

fold_out = 'detection_' + method_detection + '_' + method_landm_loc
detector = return_detector(method_detection)

In [ ]:
path_shape_pred = path_pickles + 'modelln_' +  method_detection + '_' + method_landm_loc + '.model'
predictor_dlib = shape_predictor(path_shape_pred)

In [ ]:
def process_frame(p_fr, p_ln_out, nfr, detector, landm_loc):
    im = mio.import_image(p_fr + nfr)
    if im.n_channels == 3:
        im = im.as_greyscale()
    detector(im)
    assert(im.landmarks.n_groups < 10)  # might get complicated otherwise with the naming
    im_pili = np.array(im.as_PILImage())
    ln = im.landmarks[im.landmarks.group_labels[0]]
    if landm_loc == 'dlibERT':
        det_frame = predictor_dlib(im_pili, pointgraph_to_rect(ln.lms))
        init_pc = detection_to_pointgraph(det_frame)
        mio.export_landmark_file(LandmarkGroup.init_with_all_label(init_pc), p_ln_out + 
                                 im.path.stem + '.pts', overwrite=True)

def process_clip(p_cat, p_fr, p_ln_out, clip, detector, method_landm_loc):
    assert(isdir(p_fr) and isdir(p_cat))
    list_fr = sorted(listdir(p_fr))
    assert(len(list_fr) > 100)  # random check to ensure there are frames in there
    for nfr in list_fr:
        process_frame(p_fr, p_ln_out, nfr, detector, method_landm_loc)

In [ ]:
# 'globals' used: path_base, folder_out, method_detection, method_land_loc
cats = listdir(path_base)
for cat in cats:
    if not cat[:8] == 'category' or not isdir(path_base + cat):
        warn('Unknown content in path {} (folder: {}).'.format(path_base, cat))
    if cat != 'category1': 
        continue     #################################################### DEBUGGING !!!!!!!!!!!!!!!!!!!!
    p_cat = join(path_base, cat, '')
    p_fr = join(p_cat, 'frames', '')
    assert(isdir(p_fr))  # frames folder should exist
    clips = sorted(listdir(p_fr))
    for clip in clips:
        if not isdir(p_fr + clip):
            continue
        print clip
        if clip != '114':
            continue         #################################################### DEBUGGING !!!!!!!!!!!!!!!!!!!!
        p_ln_out = mkdir_p(join(p_cat, fold_out, clip, ''))
        p_fr_cl = join(p_fr, clip, '')
        process_clip(p_cat, p_fr_cl, p_ln_out, clip, detector, method_landm_loc)

In [ ]:
im = mio.import_builtin_asset.takeo_ppm()
dlib = return_detector('ffld2')
dlib(im)

In [ ]:
print im.landmarks.group_labels
%matplotlib inline
im.view_landmarks(group='ffld2_4')